<!--
Doc Writer email@nixdabei.de
v0.0.1, 2021-03-23
-->
[Home](../index.ipynb) / Files and memory
***
# Files

## Dateien auf den Controller schreiben

[Source: docs.micropython.org](https://docs.micropython.org/en/latest/reference/filesystem.html#vfs)

1) Datei `/dir1/hello.txt` zum schreiben öffnen und `"Hello world"` reinschreiben:

In [ ]:
%serialconnect

import os

os.mkdir("/dir1") # Ordner "/dir1" anlegen ("mkdir" für "make directory")

with open("/dir1/hello.txt", "w") as f:
    f.write("Hello world!")

## Dateien auflisten

Nun den Controller **abstecken!**  
Wieder anstecken, verbinden und Dateien auflisten:  
z.B: mit

In [ ]:
%serialconnect

import os

print( os.listdir( "/dir1" ) )

oder dem "magic" `%ls`

In [ ]:
%ls /dir1

# %ls bedeutet: %ls /

## Dateiinhalt lesen

In [ ]:
print(open("/dir1/hello.txt").read())

## Dateien löschen

In [ ]:
import os
os.remove("/dir1/hello.txt")
os.rmdir( "/dir1" )

print( os.listdir( "/" ) )

## Praktisch Funktionen

In [ ]:
%serialconnect
import os

def lsDir( strDir, recurseDepth = 0, showContent = False ) :
    
    files = os.listdir(strDir)

    if strDir[-1] != "/" :
        strDir += "/"
        
    for f in files:
        fPath = strDir + f
        
        if os.stat( fPath )[0] & 0x8000 > 0: # its a file
            print( fPath )
            
            if showContent :
                print( "Content >>---" )
                print(open(fPath).read())
                print( "-----------<<\n" )
        else: # its a directory
            if   recurseDepth < 0 : lsDir( fPath, recurseDepth,    showContent )
            elif recurseDepth > 0 : lsDir( fPath, recurseDepth -1, showContent )

lsDir( "/" )
print()
lsDir( "/", -1 )
print()
lsDir( "/", 1 , True )


# Memory

## Speicherbelegung anzeigen

In [ ]:
import gc
import os

def df():
  s = os.statvfs('//')
  return ('{} MB'.format((s[0]*s[3])/1048576))

def free(bAbsolute=False):
  memFree  = gc.mem_free()
  memAlloc = gc.mem_alloc()
  memTotal = memFree + memAlloc
  return "{0:.2f}%".format(memFree/memTotal*100) if not bAbsolute else "Free: {}, Allocated: {}, Total: {}".format(memFree,memAlloc,memTotal)

def printMem():
    gc.collect() # first call garbage collector
    print( "{}; Disk: {}".format( free(True), df()) )

printMem()

## RAM-Disk anlegen

In [ ]:
%serialconnect

class RAMBlockDev:
    def __init__(self, block_size, num_blocks):
        self.block_size = block_size
        self.data = bytearray(block_size * num_blocks)

    def readblocks(self, block_num, buf, offset=0):
        addr = block_num * self.block_size + offset
        for i in range(len(buf)):
            buf[i] = self.data[addr + i]

    def writeblocks(self, block_num, buf, offset=None):
        if offset is None:
            # do erase, then write
            for i in range(len(buf) // self.block_size):
                self.ioctl(6, block_num + i)
            offset = 0
        addr = block_num * self.block_size + offset
        for i in range(len(buf)):
            self.data[addr + i] = buf[i]

    def ioctl(self, op, arg):
        if op == 4: # block count
            return len(self.data) // self.block_size
        if op == 5: # block size
            return self.block_size
        if op == 6: # block erase
            return 0

In [ ]:
import os

bdev = RAMBlockDev(512, 50)
os.VfsLfs2.mkfs(bdev)
os.mount(bdev, '/ramdisk1')


with open('/ramdisk1/hello.txt', 'w') as f:
    f.write('Hello world!!!')
print(open('/ramdisk1/hello.txt').read())


In [ ]:
%ls /ramdisk